In [ ]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import pathlib

In [ ]:
url = r"https://muppet.fandom.com/wiki/Episode_5019"

In [ ]:
# Get raw content
req = requests.get(url)

In [ ]:
# Turn Content into BS4
document = BeautifulSoup(req.content.decode(encoding="utf-8"))

In [ ]:
# Find all table elements in the document
tables = document.find_all(name='table')

In [ ]:
# Get the table document that has 
for tbl in tables:
    if tbl.find(text=re.compile('.*Picture.*')):
        res = tbl

In [ ]:
seasons = [str(season) for season in range(46,51,1)]
episodes = [(str('00')+str(i))[-2:] for i in range(1,36,1)]
from itertools import product


In [ ]:
seasons = [str(season) for season in range(46,51,1)]
episodes = [(str('00')+str(i))[-2:] for i in range(1,36,1)]
episodeList = [''.join(i) for i in product(seasons,episodes)]

episodeSummaries = []
#for episode in episodeList[:]:
episode=4715
imgName = pathlib.Path(f'C:\\Users\\riwolff\\Documents\\me_want_cookie\\data\\raw\\images\\{episode}')
imgName.mkdir(parents=True, exist_ok=True)

url = f"https://muppet.fandom.com/wiki/Episode_{episode}"
req = requests.get(url)
document = BeautifulSoup(req.content.decode(encoding="utf-8"))
tables = document.find_all(name='table')

# Get the table document that has 
for tbl in tables:
    if tbl.find(text=re.compile('.*Picture.*')):
        res = tbl
        break

txt = []
rows = res.find_all(name='tr')
i = 0
for row in rows:
    cols = row.find_all(name='td')
    rowText = [episode]
    for j, col in enumerate(cols):
        if j == 0:
                for a in col.find_all('a', href=True):
                    imgNum = '00'+str(i)
                    imgNum = imgNum[-3:] + '.jpg'
                    save_img(a['href'], imgName / imgNum)
                    i += 1
        elif j == 1:
            rowText.append(col.text.strip().replace("cont'd",''))
        else:
            wordsCleaned = [r.getText().strip() if not isinstance(r,str) else r.strip() for r in col.contents]    
            rowText.append(' '.join(wordsCleaned).strip())

    txt.append(rowText)
episodeSummaries.append(pd.DataFrame(txt,columns=['Episode','Segment','Description']))
        

In [ ]:
df = pd.concat(episodeSummaries)
df['Episode'] = df['Episode'].astype(int)
df = df[df['Episode']<5024].dropna(subset=['Segment','Description'])
df.to_csv(pathlib.Path(f'C:\\Users\\riwolff\\Documents\\me_want_cookie\\data\\raw\\seasons46to50.csv'), index=False)

In [ ]:
df[df['Episode']==4715]

In [ ]:
pic_url = 'https://vignette.wikia.nocookie.net/muppet/images/8/87/5019a.png/revision/latest?cb=20200322030436'


def save_img(pic_url, imgName):
    with open(imgName, 'wb') as handle:
            response = requests.get(pic_url, stream=True)

            if not response.ok:
                print(response)

            for block in response.iter_content(1024):
                if not block:
                    break

                handle.write(block)
    return None

In [ ]:
season = 5019
imgName = pathlib.Path(f'C:\\Users\\riwolff\\Documents\\me_want_cookie\\data\\raw\\images\\{season}')
imgName
#save_img(pic_url, path.joinpath('001.jpg'))